# OPERA RTC Validation: Point Target Absolute Geolocation Evaluation

**Alex Lewandowski & Franz J Meyer; Alaska Satellite Facility, University of Alaska Fairbanks**

This notebook analyzes the absolute geolocation quality of OPERA RTC products using corner reflectors as reference. The notebook allows for analyzing corner reflector sites in California, Oklahoma, and Alaska. 

**Notebook Notes**
- Adapted for OPERA RTCs from https://github.com/OPERA-Cal-Val/calval-CSLC/blob/dev/_ALE_.ipynb

<hr>

# 0. OPERA RTC Absolute Geolocation Requirement

<div class="alert alert-success">
<i>The Sentinel-1-based RTC product (RTC-S1) shall meet an absolute geolocation accuracy better than or equal to 6 meters given the 30 meter RTC-S1 product resolution (i.e. 20% of the product resolution), excluding the effects of DEM errors, for at least 80% of all validation products considered.</i>
</div>

<hr>

# 1. Load Necessary Libraries

In [ ]:
import asf_search
from pathlib import Path
import numpy as np
import datetime as dt
import ipywidgets as widgets
from ipywidgets import Layout
import os
import pandas as pd
import math
import scipy
import re
import requests
import warnings
import json
from pyproj import Proj, CRS
import pysolid
import matplotlib.pyplot as plt
import rasterio as rio
from isce3.core import Ellipsoid as ellips
warnings.filterwarnings('ignore')

<hr>

# 2. Download OPERA RTC Product over Validation Sites

For California Site, run this:

In [ ]:
s1_name = "S1A_IW_SLC__1SDV_20230108T135223_20230108T135251_046693_0598D3_A89F"
rtc_name = f"ISCE3_RTC_{s1_name}"
rtc_dir_path = Path.cwd()/s1_name
if not rtc_dir_path.exists():
    opera_rtc_s3_uri = f's3://asf-jupyter-data-west/OPERA_CalVal/ISCE3_RTC_samples/{rtc_name}.zip'
    !aws s3 cp "$opera_rtc_s3_uri" "$rtc_name".zip
    !unzip "$rtc_name".zip
    Path(f"{rtc_name}.zip").unlink()

For Oklahoma Site, run this:

In [ ]:
s1_name = "S1A_IW_SLC__1SDV_20230309T015038_20230309T015106_047561_05B60F_D3B2"
rtc_name = f"ISCE3_RTC_{s1_name}"
rtc_dir_path = Path.cwd()/s1_name



In [ ]:
rtc_h5_path = list(rtc_dir_path.rglob(f"ISCE3_RTC/*.h5"))[0]
rtc_geotiff_paths = list(rtc_dir_path.rglob(f"ISCE3_RTC/*_V*.tif"))
rtc_geotiff_paths.sort()
rtc_geotiff_paths

In [ ]:
with rio.open(rtc_geotiff_paths[0]) as ds:
    rtc = ds.read(1)

Reading required file parameters

In [ ]:
with rio.open(rtc_geotiff_paths[0]) as ds:   
    start_x = ds.transform[2]
    start_y = ds.transform[5]
    spacing_x = ds.transform[0]
    spacing_y = ds.transform[4]
    width = ds.profile['width']
    height = ds.profile['height']
    epsg_no = ds.crs.to_epsg()
    b = ds.bounds

**Visualizing RTC Image**

In [ ]:
# Visualize Opera Data
%matplotlib widget

fig, ax = plt.subplots(1, 1, figsize=(15, 16))

ax.set_title(rtc_geotiff_paths[0].stem)
ax.imshow(20*np.log10(np.abs(rtc)), cmap='gray',interpolation=None, origin='upper')

<hr>

# 3. Get Corner Reflector Data and Confirm RTC Coverage

Downloading Corner Reflector information from UAVSAR server

In [ ]:
date_ = re.search("\d{8}T\d{6}", rtc_geotiff_paths[0].stem).group(0)
date_ = dt.datetime.strptime(date_, '%Y%m%dT%H%M%S').strftime('%Y-%m-%d+%H\u0021%M')


# Download corner reflector data from NISAR
res = requests.get(f'https://uavsar.jpl.nasa.gov/cgi-bin/corner-reflectors.pl?date={str(date_)}&project=uavsar')
open('crdata.csv', 'wb').write(res.content)

# Read to pandas dataframe and rename columns
df = pd.read_csv('crdata.csv')
df.rename(columns={'Corner reflector ID':'ID'}, inplace=True)
df.rename(columns={'Latitude (deg)':'lat'}, inplace=True) 
df.rename(columns={'Longitude (deg)':'lon'}, inplace=True) 
df.rename(columns={'Azimuth (deg)':'azm'}, inplace=True)
df.rename(columns={'Height above ellipsoid (m)':'hgt'}, inplace=True) 
df.rename(columns={'Side length (m)':'slen'}, inplace=True)

df.head()

Discovering which Corner Reflectors are within RTC coverage:

In [ ]:
import shapely.wkt
from shapely import geometry

wkt_border = f'POLYGON(({b.left} {b.top},{b.right} {b.top},{b.right} {b.bottom},{b.left} {b.bottom},{b.left} {b.top}))'
poly = shapely.wkt.loads(wkt_border)

#calculating the locations of CRs in SAR image
UTMx = []
UTMy = []
xloc = []
yloc = []
xloc_float = []
yloc_float = []
_in = []

for idx, row in df.iterrows():
    
    _Proj = Proj(CRS.from_epsg(epsg_no))
    _x, _y = _Proj(row['lon'], row['lat'],inverse=False)     #conversion of lat/lon of CRs to UTM coordinates

    
    #location of CRs in SLC image
    _xloc = int((_x-start_x)/spacing_x)    
    _yloc = int((_y-start_y)/spacing_y)
    
    UTMx.append(_x) 
    UTMy.append(_y)
    xloc.append(_xloc)
    yloc.append(_yloc)
    xloc_float.append((_x-start_x)/spacing_x)
    yloc_float.append((_y-start_y)/spacing_y)
    _in.append(poly.contains(geometry.Point(_x, _y)))
    
df['UTMx'] = UTMx
df['UTMy'] = UTMy
df['xloc'] = xloc
df['yloc'] = yloc
df['xloc_float'] = xloc_float
df['yloc_float'] = yloc_float
df['inPoly'] = _in

#checking whether CRs are in RTC coverage. Including only CRs within RTC image
df = df[df['inPoly']==True]
df.drop('inPoly', axis=1, inplace=True)
df = df.reset_index(drop=True)

In [ ]:
df

**Visualizing** CRs on RTC Image. We color code by reflector size.

In [ ]:
#Displaying RTC image
buffer = 50
minX = df['xloc'].min() - buffer
maxX = df['xloc'].max() + buffer
minY = df['yloc'].min() - buffer
maxY = df['yloc'].max() + buffer

scale_ = 1.0
exp_ = 0.15

fig, ax = plt.subplots(figsize=(15, 7))
cax = ax.imshow(scale_*(np.abs(rtc))**exp_, cmap='gray',interpolation=None, origin='upper')
ax.set_xlim(minX,maxX)
ax.set_ylim(minY,maxY)
#ax.axis('off')

for sl in pd.unique(df.slen):
    xx = df.loc[df['slen']==sl]['xloc']
    yy = df.loc[df['slen']==sl]['yloc']
    ID = df.loc[df['slen']==sl]['ID']
    
    if sl == 2.4384:
        color='blue'
    elif sl == 4.8:
        color='red'
    elif sl == 2.8:
        color='yellow'
    else:
        color='green'
    
    ax.scatter(xx,yy,color=color,marker="+",lw=1)
    for _ID,_xx,_yy in zip(ID,xx,yy):
        ax.annotate(_ID, (_xx, _yy), fontsize=10)

ax.set_aspect(1)
fig.savefig('S1_geoRTC_CRs.png',dpi=300,bbox_inches='tight')

<hr>

# 4. Remove Corner Reflectors Facing away from the look direction of the S1 Acquisition 

In [ ]:
results = asf_search.granule_search(s1_name)
flight_direction = results[0].properties['flightDirection']
flight_direction

In [ ]:
# selecting CRs according to orbit direction
if flight_direction == 'DESCENDING':
    # descending
    df_filter = df[df['azm']>340].reset_index(drop=True)
    #only east-looking CRs (for right-looking descending)
else:
    # ascending
    df_filter = df[df['azm']<200].reset_index(drop=True)    
    #only west-looking CRs (for right-looking ascending)

df_filter = df_filter.loc[df_filter['slen']>0.8].reset_index(drop=True)   #excluding SWOT CRs (0.7 m as a side length)
df_filter

<hr>

# 5. Calculate Absolute Geolocation Error in Easting and Northing

In [ ]:
from src.ALE_utils import oversample_slc, findCR

xpeak = []
ypeak = []

for ID, xoff, yoff in zip(df_filter['ID'],df_filter['xloc'],df_filter['yloc']):
    # crop a patch of 8*8 with center at the calculated CR position
    pxbuff = 3
    pybuff = 3
    croprtc = rtc[(yoff-pybuff):(yoff+pybuff+1),(xoff-pxbuff):(xoff+pxbuff+1)]
    
    # find the peak amplitude in the 10*10 patch
    yind,xind = np.unravel_index(np.argmax(np.abs(croprtc), axis=None), croprtc.shape)
    
    # give a warning if the peak and the calculated postion are too far
    dyind = yind-pybuff; dxind = xind-pxbuff
    dist = math.sqrt(dyind**2+dxind**2)
    if dist > 5.0:
        warnings.warn(f'the most bright pixel and the xloc is too far for CR {ID}')
        
    plt.rcParams.update({'font.size': 14})
    fig, ax = plt.subplots(1, 2, figsize=(15, 7))
    ax[0].imshow(np.abs(croprtc), cmap='gray',interpolation=None, origin='upper')
    ax[0].plot(xind,yind,'r+')
    ax[0].set_title(f'Corner Reflector ID: {ID}')
    
    # crop a patch of 32*32 but with its center at the peak
    xbuff = 16
    ybuff = 16
    ycrop = np.arange(yoff+dyind-ybuff,yoff+dyind+ybuff+1)
    xcrop = np.arange(xoff+dxind-xbuff,xoff+dxind+xbuff+1)
    croprtc = rtc[ycrop,:][:,xcrop]

    # oversample this 32*32 patch by 32
    ovsFactor = 32
    croprtc_ovs,ycrop_ovs,xcrop_ovs = oversample_slc(croprtc,sampling=ovsFactor,y=ycrop,x=xcrop)

    # find the peak amplitude again in a 4 x 4 patch, it correspond to 
    # (2*ovsFactor) x (2*ovsFactor) in oversampled slc
    yoff2 = int(croprtc_ovs.shape[0]/2)
    xoff2 = int(croprtc_ovs.shape[1]/2)
    croprtc2 = croprtc_ovs[yoff2-2*ovsFactor:yoff2+2*ovsFactor+1,
                           xoff2-2*ovsFactor:xoff2+2*ovsFactor+1]
    yind2,xind2 = np.unravel_index(np.argmax(abs(croprtc2), axis=None), croprtc2.shape)
    dyind2 = yind2-ovsFactor; dxind2 = xind2-ovsFactor
    
    ax[1].imshow(np.abs(croprtc2), cmap='gray',interpolation=None, origin='upper')
    ax[1].plot(xind2,yind2,'r+')
    ax[1].set_title(f'Oversampled Corner Reflector ID: {ID}')

    # crop a patch of 3x3 oversampled patch with center at the peak
    croprtc2 = croprtc_ovs[yoff2+dyind2-1:yoff2+dyind2+2,xoff2+dxind2-1:xoff2+dxind2+2]
    ycrop2 = ycrop_ovs[yoff2+dyind2-1:yoff2+dyind2+2]
    xcrop2 = xcrop_ovs[xoff2+dxind2-1:xoff2+dxind2+2]
    xxcrop2,yycrop2 = np.meshgrid(xcrop2,ycrop2)
    xxcrop2_f = xxcrop2.flatten()
    yycrop2_f = yycrop2.flatten()
    croprtc2_f = croprtc2.flatten()

    # Check if pixel values in a patch are non-NaN
    valid = ~(np.isnan(croprtc2_f))
    count_valid = np.count_nonzero(valid)

    if count_valid == 0:
        _ypeak, _xpeak = [np.nan, np.nan]

    else:
        _ypeak,_xpeak = findCR(np.abs(croprtc2_f[valid]),yycrop2_f[valid],xxcrop2_f[valid],
                            x_bound=[xcrop2[0],xcrop2[-1]],y_bound=[ycrop2[0],ycrop2[-1]],method="para")

    xpeak.append(_xpeak)
    ypeak.append(_ypeak)

df_filter['xloc_CR'] = xpeak
df_filter['yloc_CR'] = ypeak

**Visualizing CR Location Measurements**

In [ ]:
df_filter = df_filter.dropna()
df_filter

**Calculating Absolute Geolocation Numbers:**

In [ ]:
#absloute geolocation error in range and azimuth
ALE_Rg = (df_filter['xloc_CR'] -  df_filter['xloc_float'])
ALE_Az = (df_filter['yloc_CR'] - df_filter['yloc_float'])

<hr>

# 6. Plot Absolute Geolocation Error in Easting and Northing

In [ ]:
#plotting ALE
fig, ax = plt.subplots(figsize=(8,8))
Circle1= plt.Circle(( 0.0 , 0.0 ), 6.0, fill=False)
ax.add_patch(Circle1)
sc = ax.scatter(ALE_Rg, ALE_Az, s=200, c=df_filter['slen'], alpha=0.6, marker='o')
ax.legend(*sc.legend_elements(),facecolor='lightgray')
ax.get_legend().set_title('side length (m)')


for ii, txt in enumerate(df_filter.iloc[:,0]):
    ax.annotate(txt, (ALE_Rg[ii],ALE_Az[ii]), color='black',xytext=(0, 5), textcoords='offset points')   #putting IDs in each CR
    
ax.grid(True)
ax.set_xlim(-7.25,7.25)
ax.set_ylim(-7.25,7.25)
ax.axhline(0, color='black')
ax.axvline(0, color='black')

ax.set_title(f'Easting: {np.round(np.nanmean(ALE_Rg), 3)} +/- {np.round(np.nanstd(ALE_Rg),3)} m, \
    Northing: {np.round(np.nanmean(ALE_Az),3)}, +/- {np.round(np.nanstd(ALE_Az),3)} m')
ax.set_xlabel('Easting error (m)')
ax.set_ylabel('Northing error (m)')
fig.suptitle('Absolute location error')
fig.savefig('ALE_geoSLC.png',dpi=300,bbox_inches='tight')

plt.errorbar(np.round(np.nanmean(ALE_Rg), 3), np.round(np.nanmean(ALE_Az),3),\
             xerr=np.round(np.nanstd(ALE_Rg),3), yerr=np.round(np.nanstd(ALE_Az),3), \
             barsabove=True, capsize=8, capthick=2, fmt='ro', linewidth=2, markersize=20)

output = f"{s1_name}_GeolocationPLOT.png"

plt.savefig(output, dpi=300, transparent='true')

*ALE_OPERA_RTC.ipynb - Version 2.0.0 - April 2023*

*Change log*

- Made CR discovery more robust
- Added average visualization in geolocation plot
- Made formatting changes